# wps_bccaq
A process that runs the [bccaq.netcdf.wrapper](https://github.com/pacificclimate/ClimDown/blob/master/R/BCCAQ.R) function from ClimDown.

Bias Correction/Constructed Analogues with Quantile mapping reordering (BCCAQ) is composed of the following steps.
- Constructed Analogues (CA)
- Climate Imprint (CI)
- Quantile Delta Mapping (QDM)
- Rerank


In [ ]:
from birdy import WPSClient
from netCDF4 import Dataset
from pkg_resources import resource_filename
from wps_tools.testing import get_target_url
from wps_tools.utils import copy_http_content
from tempfile import NamedTemporaryFile
from datetime import date
import requests
import os

# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "chickadee":
    os.chdir('../')

In [ ]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("chickadee")
print(f"Using chickadee on {url}")

In [ ]:
chickadee = WPSClient(url)

### Help for individual processes can be diplayed using the ? command (ex. bird.process?).

In [ ]:
# NBVAL_IGNORE_OUTPUT
chickadee.bccaq?

### Run the bccaq process

In [ ]:
out_file = NamedTemporaryFile(suffix=".nc", prefix="output_", dir="/tmp", delete=True)

output = chickadee.bccaq(
    gcm_file=resource_filename("tests", "/data/tiny_gcm.nc"), 
    obs_file=resource_filename("tests", "/data/tiny_obs.nc"), 
    varname="tasmax", 
    end_date=date(1972, 12, 31),
    out_file=out_file.name,
    num_cores=4
)

out_file.close()

output_data = output.get()[0]

### Test for expected output

In [ ]:
with NamedTemporaryFile(suffix=".nc", prefix="tmp_copy", dir="/tmp", delete=True) as tmp_file:
    output_data = Dataset(copy_http_content(output_data, tmp_file))
    expected_data = Dataset(resource_filename("tests","/data/bccaq_expected_output.nc"))
    for key, value in expected_data.dimensions.items():
        assert str(output_data.dimensions[key]) == str(value)